In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from bidi.algorithm import get_display
import arabic_reshaper
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.isri import ISRIStemmer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge 
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from sklearn.metrics import classification_report



from sklearn.preprocessing import LabelEncoder  

from wordcloud import WordCloud 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('data_mr.cvs', index_col=0)

In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 350
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          351 non-null    object
 1   court          351 non-null    object
 2   court_id       351 non-null    object
 3   court_date     351 non-null    object
 4   appeal_court   307 non-null    object
 5   appeal_id      307 non-null    object
 6   appeal_date    274 non-null    object
 7   region         274 non-null    object
 8   city           318 non-null    object
 9   judgment_text  351 non-null    object
 10  appeal_text    274 non-null    object
 11  appeal_data    33 non-null     object
 12  appleal_text   33 non-null     object
 13  Verdict        351 non-null    object
 14  incident       351 non-null    object
 15  court_y        351 non-null    object
dtypes: object(16)
memory usage: 54.7+ KB


In [4]:
data.head(3)

,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text,appeal_data,appleal_text
0,القضية رقم ٤٦٤٦ لعام ١٤٤٢ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٤٦٤٦,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٨٥,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
1,القضية رقم ٦٩٥٨ لعام ١٤٤٢ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٦٩٥٨,تاريخها: ٢٥/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٤٦,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
2,القضية رقم ٦٦٠٧ لعام ١٤٤٢ هـ,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٦٦٠٧,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٩٠,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN


# EDA

# Data Cleaning


In [5]:
data['court'] =data['court'].str.replace('المحكمة: المحكمة التجارية', 'التجارية')
data['court'] =data['court'].str.replace('المحكمة: المحكمة العليا -  الهيئة الدائمة', 'العليا')
data['court'] =data['court'].str.replace('المحكمة: المحكمة العامة', 'العامة')

In [6]:
data.court.value_counts()

التجارية             166
 المحكمة التجارية    131
العليا                39
العامة                 8
 المحكمة العامة        7
Name: court, dtype: int64

In [7]:
data.court.value_counts()

التجارية             166
 المحكمة التجارية    131
العليا                39
العامة                 8
 المحكمة العامة        7
Name: court, dtype: int64

In [8]:
data[data['judgment_text'].isnull()]

,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text,appeal_data,appleal_text


In [9]:
data.dropna(subset=["judgment_text"],inplace = True)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 350
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          351 non-null    object
 1   court          351 non-null    object
 2   court_id       351 non-null    object
 3   court_date     351 non-null    object
 4   appeal_court   307 non-null    object
 5   appeal_id      307 non-null    object
 6   appeal_date    274 non-null    object
 7   region         274 non-null    object
 8   city           318 non-null    object
 9   judgment_text  351 non-null    object
 10  appeal_text    274 non-null    object
 11  appeal_data    33 non-null     object
 12  appleal_text   33 non-null     object
dtypes: object(13)
memory usage: 38.4+ KB


In [11]:

# data.drop(['نص الحكم','links1','links0-href','links0','appeal_text','appeal_date','appeal_id','appeal_court','court_date',
#           'court_id','links1-href'], axis = 1,inplace=True)
#           '''

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 350
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          351 non-null    object
 1   court          351 non-null    object
 2   court_id       351 non-null    object
 3   court_date     351 non-null    object
 4   appeal_court   307 non-null    object
 5   appeal_id      307 non-null    object
 6   appeal_date    274 non-null    object
 7   region         274 non-null    object
 8   city           318 non-null    object
 9   judgment_text  351 non-null    object
 10  appeal_text    274 non-null    object
 11  appeal_data    33 non-null     object
 12  appleal_text   33 non-null     object
dtypes: object(13)
memory usage: 38.4+ KB


In [13]:
#data['الصفحات-href'].isnull().sum()

In [14]:
#data['القضايا-href'].isnull().sum()

In [15]:
#data['pages-href'].isnull().sum()

In [16]:
#data['pages'].isnull().sum()

In [17]:
#data.drop(['pages','pages-href','القضايا-href','الصفحات-href'], axis = 1,inplace=True)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 350
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          351 non-null    object
 1   court          351 non-null    object
 2   court_id       351 non-null    object
 3   court_date     351 non-null    object
 4   appeal_court   307 non-null    object
 5   appeal_id      307 non-null    object
 6   appeal_date    274 non-null    object
 7   region         274 non-null    object
 8   city           318 non-null    object
 9   judgment_text  351 non-null    object
 10  appeal_text    274 non-null    object
 11  appeal_data    33 non-null     object
 12  appleal_text   33 non-null     object
dtypes: object(13)
memory usage: 38.4+ KB


In [19]:
#data['القضايا'].isnull().sum()

In [20]:
#data['الصفحات'].isnull().sum()

In [21]:
#data.drop(['القضايا','الصفحات'], axis = 1,inplace=True)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 350
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          351 non-null    object
 1   court          351 non-null    object
 2   court_id       351 non-null    object
 3   court_date     351 non-null    object
 4   appeal_court   307 non-null    object
 5   appeal_id      307 non-null    object
 6   appeal_date    274 non-null    object
 7   region         274 non-null    object
 8   city           318 non-null    object
 9   judgment_text  351 non-null    object
 10  appeal_text    274 non-null    object
 11  appeal_data    33 non-null     object
 12  appleal_text   33 non-null     object
dtypes: object(13)
memory usage: 38.4+ KB


In [23]:
data.head(4)

,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text,appeal_data,appleal_text
0,القضية رقم ٤٦٤٦ لعام ١٤٤٢ هـ,التجارية,رقم القضية - القرار: ٤٦٤٦,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٨٥,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
1,القضية رقم ٦٩٥٨ لعام ١٤٤٢ هـ,التجارية,رقم القضية - القرار: ٦٩٥٨,تاريخها: ٢٥/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٤٦,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
2,القضية رقم ٦٦٠٧ لعام ١٤٤٢ هـ,التجارية,رقم القضية - القرار: ٦٦٠٧,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٩٠,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
3,القضية رقم ٤٣٣١ لعام ١٤٤٢ هـ,التجارية,رقم القضية - القرار: ٤٣٣١,تاريخها: ١٨/١١/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٣٧,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN


## remove ponctuations

In [24]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|"!”…“–ـ'''

In [25]:
def remove_punctuations(text):
    translator = str.maketrans(' ', ' ', arabic_punctuations)
    return text.translate(translator)
data['judgment_text'] =data['judgment_text'].map(remove_punctuations)

In [26]:
#data.judgment_text = data.judgment_text.replace('\n','')

## normalize_arabic

In [27]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [28]:
data['judgment_text'] =data['judgment_text'].map(normalize_arabic)

## remove numbers

In [29]:
data['judgment_text'] =data['judgment_text'].str.replace('\w*\d\w*', ' ')

In [30]:
data['judgment_text'] =data['judgment_text'].str.replace('\\', ' ')

## remove \n

In [31]:
data['judgment_text'] =data['judgment_text'].str.replace(r'\s', ' ')

In [32]:
data['judgment_text'] =data['judgment_text'].str.replace(r'\s*[A-Za-z]+\b', ' ')

In [33]:
data['judgment_text'] =data['judgment_text'].str.replace('-', ' ')

In [34]:
data['judgment_text'] =data['judgment_text'].str.replace('\u200c', ' ')

In [35]:
data['judgment_text'][1]

'الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد فلدي الدائره التجاريه الثانيه عشره وبناءً علي القضيه رقم    لعام   ه  المقامه من شركه شبال للتجاره مساهمه مقفله سجل تجاري   ضد عبدﷲ محمد عبدﷲ فلاته هويه وطنيه    القاضي بندر بن صالح الحميد رئيسا القاضي بكر بن خلف العنزي عضوا القاضي عبدالرحمن بن سعد السعد عضوا   الوقائع  تتحصل وقائع هذه الدعوي بالقدر اللازم لاصدار هذا الحكم وحسبما يتبين من اوراقها بتقدم وكيل المدعي نايف بدر غازي المرشدي هويه رقم  بصفته وكيلا عن شركه شبال للتجاره مساهمه مقفله بوكاله رقم    بلائحه دعوي ضد المدعي عليها مؤسسه اثير الصفوه للتجاره سجل رقم  تضمنت ما يلي    بتاريخ       م قامت المدعي عليها بفتح حساب شراء بضاعه بالاجل لدي موكلتي علي ان توفر موكلتي للمدعي عليها الاقمشه المطلوبه بالاجل وتقوم المدعي عليها بالسداد خلال العام نفسه ولا يتم تجاوز نفس العام    استمرت العلاقه التعاقديه طيله الفتره الماضيه بناءً علي الاتفاق نفسه مع تغيير المؤسسه المباشره للتعاملات الي مؤسسه اثير الصفوه للتجاره والعائده جميعها للمالك نفسه عبدﷲ محمد عبدﷲ فلاته    اشترت المدعي عليها اقمشه رجاله ص

Strip tatweel from a text and return a result text

In [36]:
import pyarabic.araby as araby
import pyarabic.number as number

In [37]:
#araby.strip_tashkeel(data['judgment_text'])

In [38]:
'''
def converter(x):
    try:
        return ([araby.strip_tashkeel(x) for i in x])
    except AttributeError:
        return None  # or some other value
data['judgment_text'] = data['judgment_text'].str.apply(converter)
'''

"\ndef converter(x):\n    try:\n        return ([araby.strip_tashkeel(x) for i in x])\n    except AttributeError:\n        return None  # or some other value\ndata['judgment_text'] = data['judgment_text'].str.apply(converter)\n"

In [39]:
'''
def remove_tashkeel(f):
    return araby.strip_tashkeel(f)
data['judgment_text'] = data.judgment_text.map(remove_tashkeel)
'''

"\ndef remove_tashkeel(f):\n    return araby.strip_tashkeel(f)\ndata['judgment_text'] = data.judgment_text.map(remove_tashkeel)\n"

In [40]:
data.judgment_text = data.judgment_text.str.replace('\u0640', '')

In [41]:
data.judgment_text= data.judgment_text.str.replace('\u064E', '')# َ

In [42]:
data.judgment_text= data.judgment_text.str.replace('\u0650', '')# َ

In [43]:
 def get_Verdict(Verdict):
        
    if "حكمت"  in Verdict:
         return str.split(Verdict, "حكمت",1)[1]
    #if "حـكمـت"  in Verdict:
        # return str.split(Verdict, "حـكمـت",1)[1]
    if "قررت"  in Verdict:
         return str.split(Verdict, "قررت",1)[1]
    if "منطوق"  in Verdict:
         return str.split(Verdict, "منطوق",1)[1]
    if "منطوف"  in Verdict:
         return str.split(Verdict, "منطوف",1)[1]
    return None
data['Verdict'] =data['judgment_text'].map(get_Verdict)

In [44]:
data['Verdict'][200]

' الدائره رفع الجلسه للمداوله اصدار القرار   الاسباب لما كان وكيل المدعيات يطلب تصحيح الحكم الوارد في هذه القضيه ليكون الحكم لموكلته ولان ما طلبه وكيل المدعي عليه لا يعد خطا يلزم منه التصحيحلان تغير المحكوم عليه من شركه الي فرد يعني تغير المحكوم عليه دون اجراء مرافعهلان النظام اعتبر الشركه شخصاً مختلفاً عن مالكها ولها ذمه ماليه مستقله كما انه قد تكون ثمت مطالبات علي الشركه اولي بالسداد من الشركاء كما ان احد المدعي عليهما من ضمن الشركاء المحكوم عليه في هذه القضيهمما تنتهي الدائره الي ما يرد في منطوقه   منطوق الحكم قررت الدائره  رفض تصحيح الحكم المقدم من وكيل المدعيات  عضو  عبدﷲ بن محمد البدر  عضو  يوسف بن محمد بن قاسم جعفري رئيس الدائره جمال بن فاضل العمري'

In [45]:
#data['judgment_text'][29]

In [46]:
blob = TextBlob("منطوف")
print(blob.correct())

منطوف


In [47]:
data[data['Verdict'].isnull()]

,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text,appeal_data,appleal_text,Verdict


In [48]:
'''
for col in data['Verdict']:
    data['Verdict'].to_csv('Verdict.csv')
    '''

"\nfor col in data['Verdict']:\n    data['Verdict'].to_csv('Verdict.csv')\n    "

In [49]:
'''
for col in data['judgment_text']:
    data['judgment_text'].to_csv('judgment_csv')
    '''

"\nfor col in data['judgment_text']:\n    data['judgment_text'].to_csv('judgment_csv')\n    "

In [50]:
'''
with open("judgment_csv") as f1:
    set1 = set(f1.readlines())
with open("Verdict.csv") as f2:
    set2 = set(f2.readlines())

nondups = set1 - set2

with open("file1.csv", "w") as out:
    out.writelines(nondups)
    '''

'\nwith open("judgment_csv") as f1:\n    set1 = set(f1.readlines())\nwith open("Verdict.csv") as f2:\n    set2 = set(f2.readlines())\n\nnondups = set1 - set2\n\nwith open("file1.csv", "w") as out:\n    out.writelines(nondups)\n    '

In [51]:
#txt = pd.read_csv('file1.csv')

In [52]:
data['judgment_text'][88]

'الحمد لله وحده والصلاه والسلام علي نبينا محمد وعلي اله وصحبه وبعد  ففي يوم الاثنين   ه انعقدت الدائره الخامسه بالمحكمه العليا المشكله من    عبدالوهاب بن محمد المنصوري رئيساً  عبدالله بن عبدالرحيم الزهراني عضواً  محمد بن صالح اليحيي عضواً   وذلك للنظر في طلب النقض المقدّم من شركه  للصناعه رقم السجل التجاري  علي حكم دائره الاستئناف الاولي بالمحكمه التجاريه بجده رقم   في   في القضيه المقامه منهاضد  وبعد الاطلاع علي القضيه الوارده من المحكمه التجاريه بجده رقم   وتاريخ   والمحاله لهذه الدائره بتاريخ   ودراستها وبعد المداوله وحيث انه قبل الدخول في الموضوع يتعين معرفه مدي جواز الطعن علي الحكم الصادر من الاستئناف من عدمه وفي بيان ذلك فان الماده   من نظام المحاكم التجاريه قد نصت علي انه  لا يقبل الحكم النهائي الصادر في اختصاص المحكمه نوعيًّا او مكانيًّا الاعتراض باي طريق  اه وحيث ان الحكم محل الطعن قد قضي بعدم اختصاص المحاكم التجاريه نوعياً الامر الذي تنتهي معه الدائره الي عدم جواز نظر الاعتراض لعدم قابليه الحكم للطعن وتامر بعدم القبول اعمالاً للماده   من نظام المحاكم التجاريه   لذلك قررت الدا

In [53]:
data['Verdict'][88]

' الدائره عدم قبول الاعتراض لما هو مبين بالاسباب اعلاه وبالله التوفيق وصلي الله علي نبينا محمد وعلي اله وصحبه وسلم  عضو محمد بن صالح اليحيي  عضو عبدالله بن عبدالرحيم الزهراني  رئيس الدائره عبدالوهاب بن محمد المنصوري'

In [54]:
 def get_incident(incident):
        
    if "حكمت"  in incident:
         return str.split(incident, "حكمت",1)[0]
    #if "حـكمـت"  in incident:
        # return str.split(incident, "حـكمـت",1)[0]
    if "قررت"  in incident:
         return str.split(incident, "قررت",1)[0]
    if "منطوق"  in incident:
         return str.split(incident, "منطوق",1)[0]
    if "منطوف"  in incident:
         return str.split(incident, "منطوف",1)[0]
    return None
data['incident'] =data['judgment_text'].map(get_incident)

In [55]:
data['incident'][88]

'الحمد لله وحده والصلاه والسلام علي نبينا محمد وعلي اله وصحبه وبعد  ففي يوم الاثنين   ه انعقدت الدائره الخامسه بالمحكمه العليا المشكله من    عبدالوهاب بن محمد المنصوري رئيساً  عبدالله بن عبدالرحيم الزهراني عضواً  محمد بن صالح اليحيي عضواً   وذلك للنظر في طلب النقض المقدّم من شركه  للصناعه رقم السجل التجاري  علي حكم دائره الاستئناف الاولي بالمحكمه التجاريه بجده رقم   في   في القضيه المقامه منهاضد  وبعد الاطلاع علي القضيه الوارده من المحكمه التجاريه بجده رقم   وتاريخ   والمحاله لهذه الدائره بتاريخ   ودراستها وبعد المداوله وحيث انه قبل الدخول في الموضوع يتعين معرفه مدي جواز الطعن علي الحكم الصادر من الاستئناف من عدمه وفي بيان ذلك فان الماده   من نظام المحاكم التجاريه قد نصت علي انه  لا يقبل الحكم النهائي الصادر في اختصاص المحكمه نوعيًّا او مكانيًّا الاعتراض باي طريق  اه وحيث ان الحكم محل الطعن قد قضي بعدم اختصاص المحاكم التجاريه نوعياً الامر الذي تنتهي معه الدائره الي عدم جواز نظر الاعتراض لعدم قابليه الحكم للطعن وتامر بعدم القبول اعمالاً للماده   من نظام المحاكم التجاريه   لذلك '

In [56]:
data[data['incident'].isnull()]

,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text,appeal_data,appleal_text,Verdict,incident


## Stemming

In [57]:
st = ISRIStemmer()
def streem(f):
    return st.stem(f)
#data['Verdict'] = data['Verdict'].map(streem)
data['incident'] = data['incident'].map(streem)

In [58]:
data['Verdict'][10]

' الدائره  بعدم اختصاص المحاكم التجاريه نوعياً بنظر هذه الدعوي وهي من اختصاص المحكمه العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث'

# Stop words 

In [59]:
stopwords = nltk.corpus.stopwords.words('arabic')

In [60]:
'و' in stopwords

True

In [61]:
word_st = 'الحمد','لله','تعالي','التوفيق','وصلي','الله','وسلم','علي','نبينا','محمد','اله','وصحبه','اجمعين','عضو','رئيس','الدائره','المحكمه','المحاكم','الحمد','والصلاه','والسلام','القضيه','رقم','لعام','سجل','القاضي','الموافق','المدعي','وكيل','الوكاله','المحاماه','المحاكم','بن','محكمة'

In [62]:
for i in range(len(word_st)):
    stopwords.append(word_st[i])

In [63]:
word_st2 = 'رسول','ﷲ','و','العقد','الحكم','الدعوي','للمحاكم','منطوقه','منطوق'

In [64]:
for i in range(len(word_st2)):
    stopwords.append(word_st2[i])

In [65]:
#data['Verdict'] = data['Verdict'].apply(lambda t: " ".join(word for word in t.split() if word not in stopwords))

In [66]:
data['Verdict'][100]

' الدائره  بالزام المدعي عليها شركه عماد للتجاره والمقاولات لاصحابها صالح غرم ﷲ ابوعدله الغامدي واولاده   سجل تجاري رقم  بان تدفع للمدعيه مصنع بن زقر للمواد العازله   سجل تجاري رقم  مبلغاً قدره مائه وثلاثه وسبعون الفاً وسبعمائه وسبعه وستون ريالاً    عضو   معاذ بن حمد بن محمد الزيدان  عضو   معاذ بن سامي بن عبدالعزيز العبدالقادر رئيس الدائره  مالك بن محمد بن عبداللطيف الملحم'

In [67]:
data['incident'] = data['incident'].apply(lambda t: " ".join(word for word in t.split() if word not in stopwords))

In [68]:
data['incident'][70]

'حمد اما فلدي التجاريه الحاديه وبناء المقامه شركه بنوه للمقاولات تجاري ضد شركه الخليج للتاثيث المحدوده عبدالرحيم بندر العثمان رئيسا الوقائع تتلخص وقائع يكفي لاصدار ان وكاله تقدم الي بصحيفه دعوي جاء نصه اتفقت شركه بنوه شركه الخليج توريد وتركيب قواطع لكليه الهندسه بجامعه حائل وذلك بتاريخ وقد استلمت عليها المدعيه دفعه مقدمه قدرها اربعمائه وسبع وستون الفا وستمائه وسبع وثلاثون ريالا وقد تاخرت عليها تنفيذ التزاماتها ولكون صاحبه المشروع جامعه حائل استبعدت عليها فقد تعهدت عليها باعاده الدفعه المقدمه المسلمه دفعات تاخير او مماطله الا ان عليها تسدد الدفعه المسلمه سوي مبلغ مائتين وخمسين الف وتبقي ذمتها مبلغ مائتين وسبعه الفا وستمائه وسبع وثلاثون لذا نطلب يلي الزام عليها بسداد المبلغ المذكور الزام عليها بسداد بمبلغ قدره الف مقابل اتعاب ادعي وارفق ضمن الصحيفه يراه سندا لدعواه عباره المبرم الطرفين والمؤرخ الشيك المسحوب مصرف الراجحي برقم وتاريخ وقدره اربعمائه وسبع وستون الفا وستمائه وسبع وثلاثون ريالا الشيك المسحوب البنك الاهلي برقم وتاريخ وقدره مائه الف سندي القبض الصادره المدعيه برقم وتاريخ ورقم وت

In [69]:
data['Verdict'][10]

' الدائره  بعدم اختصاص المحاكم التجاريه نوعياً بنظر هذه الدعوي وهي من اختصاص المحكمه العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث'

In [70]:
result = data['Verdict'].str.split('اختصاص المحكمه',expand=True)[1]

In [71]:
result.value_counts()

 العامه عضو يوسف محمد قاسم جعفري عضو بندر حمد التويجري رئيس الدائره عبدالعزيز سليمان الغيث                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [72]:
data['Verdict']

0       الدائره برفض الدعوي المقامه من شركه بيت الجزي...
1       الدائره بعدم قبول هذه الدعوي وﷲ الموفق وصلي ﷲ...
2       الدائره غيابيا بالزام خالد جمعه رضا حبش سجل م...
3       الدائره  برفض هذه الدعوي   عضو عبدالعزيز بن س...
4       الدائره بعدم جواز نظر هذه الدعوي لما هو مبين ...
                             ...                        
346     الدائره يلي الزام المدعي شركه مجموعه الراجحي ...
347     الدائره رفعت الجلسه للمداوله والحكم الاسباب ب...
348     صدق الحكم محكمه الاستئناف بتاريخ  اقام موكلي ...
349     حكمها ادناه منطوق الحكم الزام المدعي شركه دره...
350     الدائره برفض الدعوي عضو عبدالعزيز سليمان الغي...
Name: Verdict, Length: 351, dtype: object

In [73]:
data.court[3]

'التجارية'

In [74]:
'''
def get_right_court(index, i):
    
    #for i in data['Verdict']:
    if "بعدم اختصاص"  in i:
            #data['right_court'] = data['Verdict'].str.split("من اختصاص المحكمه",1)[0]  
        if "من اختصاص المحكمه" in i:
            return str.split(i, "من اختصاص المحكمه",0)[0]
        else:
            return 2#None
    else:#if "اختصاص" not in i:
        return  data.loc[data.court[index]]


data['right_court'] =data['Verdict'].apply(lambda p: get_right_court(p, p.index))
'''

'\ndef get_right_court(index, i):\n    \n    #for i in data[\'Verdict\']:\n    if "بعدم اختصاص"  in i:\n            #data[\'right_court\'] = data[\'Verdict\'].str.split("من اختصاص المحكمه",1)[0]  \n        if "من اختصاص المحكمه" in i:\n            return str.split(i, "من اختصاص المحكمه",0)[0]\n        else:\n            return 2#None\n    else:#if "اختصاص" not in i:\n        return  data.loc[data.court[index]]\n\n\ndata[\'right_court\'] =data[\'Verdict\'].apply(lambda p: get_right_court(p, p.index))\n'

In [75]:
'''
def get_right_court(index, i):
    
    #for i in data['Verdict']:
    if "بعدم اختصاص"  in i:
            #data['right_court'] = data['Verdict'].str.split("من اختصاص المحكمه",1)[0]  
        if "من اختصاص المحكمه" in i:
            return str.split(i, "من اختصاص المحكمه",0)[0]
        else:
            return 2#None
    else:#if "اختصاص" not in i:
        return  data.loc[data.court[index]]


data['right_court'] =data['Verdict'].apply(lambda p,index: get_right_court(p, p.index))
'''

'\ndef get_right_court(index, i):\n    \n    #for i in data[\'Verdict\']:\n    if "بعدم اختصاص"  in i:\n            #data[\'right_court\'] = data[\'Verdict\'].str.split("من اختصاص المحكمه",1)[0]  \n        if "من اختصاص المحكمه" in i:\n            return str.split(i, "من اختصاص المحكمه",0)[0]\n        else:\n            return 2#None\n    else:#if "اختصاص" not in i:\n        return  data.loc[data.court[index]]\n\n\ndata[\'right_court\'] =data[\'Verdict\'].apply(lambda p,index: get_right_court(p, p.index))\n'

In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 350
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          351 non-null    object
 1   court          351 non-null    object
 2   court_id       351 non-null    object
 3   court_date     351 non-null    object
 4   appeal_court   307 non-null    object
 5   appeal_id      307 non-null    object
 6   appeal_date    274 non-null    object
 7   region         274 non-null    object
 8   city           318 non-null    object
 9   judgment_text  351 non-null    object
 10  appeal_text    274 non-null    object
 11  appeal_data    33 non-null     object
 12  appleal_text   33 non-null     object
 13  Verdict        351 non-null    object
 14  incident       351 non-null    object
dtypes: object(15)
memory usage: 52.0+ KB


In [77]:
conditions1 = [
    (data['court'] == "التجارية"),
    (data['court'] == "العليا"),
    (data['court'] == "العامة"),
   ]
values1 = ['3','2','1']  
data['court_y'] = np.select(conditions1,values1)

In [78]:
data['court']

0               التجارية
1               التجارية
2               التجارية
3               التجارية
4               التجارية
             ...        
346     المحكمة التجارية
347     المحكمة التجارية
348     المحكمة التجارية
349       المحكمة العامة
350     المحكمة التجارية
Name: court, Length: 351, dtype: object

# spliting data

In [79]:
y=data['court_y']
X=data['judgment_text']

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [81]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2)

In [82]:
'''
X_val.values.reshape(-1,1)
X_train.values.reshape(-1,1)
y_train.values.reshape(-1,1)
'''

'\nX_val.values.reshape(-1,1)\nX_train.values.reshape(-1,1)\ny_train.values.reshape(-1,1)\n'

In [83]:
from collections import Counter
counter = Counter(y_train)
for k, v in counter.items():
    dist = v / len(y) * 100 
    print(f"Class= {k}, n={v} ({dist}%)")

Class= 3, n=95 (27.065527065527068%)
Class= 1, n=5 (1.4245014245014245%)
Class= 0, n=88 (25.071225071225072%)
Class= 2, n=22 (6.267806267806268%)


# Create Bag of words 

In [84]:
''''
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 3))
'''

"'\nfrom sklearn.feature_extraction.text import CountVectorizer\ncv = CountVectorizer(ngram_range=(1, 3))\n"

In [85]:
#cv.fit(X_train)

In [86]:
#len(cv.get_feature_names())

In [87]:
#X_train_cv = cv.transform(X_train)
#X_val_cv = cv.transform(X_val)

In [88]:
#X_train_cv

# smote

In [90]:

# from imblearn.over_sampling import SMOTE
# oversample = SMOTE(random_state = 0)
# X_train_cv, y_train = oversample.fit_resample(X_train_cv, y_train)
# ''''

In [ ]:

# from collections import Counter
# counter = Counter(y_train)
# for k, v in counter.items():
#     dist = v / len(y) * 100 
#     print(f"Class= {k}, n={v} ({dist}%)")
 

# TF/IDF

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv_tfidf = TfidfVectorizer()

In [92]:
X_train_tf = cv_tfidf.fit_transform(X_train)
X_val_tf = cv_tfidf.transform(X_val)

In [93]:
X_train_tf

<210x16016 sparse matrix of type '<class 'numpy.float64'>'
	with 100756 stored elements in Compressed Sparse Row format>

#  Logistic Regression 

In [94]:
lgg = LogisticRegression()
lgg.fit(X_train_tf, y_train)
print('Logistic Regression with TF/IDF')
lgg.score(X_train_tf, y_train)

Logistic Regression with TF/IDF


0.9714285714285714

In [95]:
y_pred_tf = lgg.predict(X_val_tf)
lgg.score(X_val_tf, y_val)

0.9285714285714286

In [96]:
print(classification_report(y_val, y_pred_tf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       0.00      0.00      0.00         1
           2       1.00      0.56      0.71         9
           3       0.87      1.00      0.93        34

    accuracy                           0.93        70
   macro avg       0.72      0.64      0.66        70
weighted avg       0.92      0.93      0.92        70



# Decision Tree

In [97]:
de = DecisionTreeClassifier(max_depth=4)
de.fit(X_train_tf, y_train)
print('Decision Tree with TF/IDF')
de.score(X_train_tf, y_train)

Decision Tree with TF/IDF


1.0

In [98]:
y_pred_tf = de.predict(X_val_tf)
de.score(X_val_tf, y_val)

0.9571428571428572

In [99]:
print(classification_report(y_val, y_pred_tf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       0.33      1.00      0.50         1
           2       1.00      0.89      0.94         9
           3       0.97      0.94      0.96        34

    accuracy                           0.96        70
   macro avg       0.83      0.96      0.85        70
weighted avg       0.98      0.96      0.96        70



# ____________________________________

# EDA

In [ ]:
d = data['judgment_text'].str.cat(sep=' ')
plt.figure(figsize=(12,10))
wordcloud_neg = WordCloud(max_font_size=200, background_color="white").generate(d) 
plt.imshow(wordcloud_neg, interpolation='bilinear')
plt.axis('off')
plt.title("");

In [ ]:
zlbl= get_display( arabic_reshaper.reshape('عدد القضايا في محاكم وزارة العدل'))
plt.figure(figsize=(10,5))
plt.title(zlbl)
df_tuple =list(zip(data['court']))
first_columns= [get_display( arabic_reshaper.reshape(x[0])) for x in df_tuple]
xlbl = get_display( arabic_reshaper.reshape(' أنواع المحاكم '))
sns.countplot(x=first_columns, data=data, color='green');
plt.xlabel(xlbl, fontdict=None, labelpad=None)
plt.tight_layout()
plt.show()